In [ ]:
#Mounting the google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#https://github.com/berenslab/rna-seq-tsne/blob/master/demo.ipynb

# Prepare

%matplotlib notebook

import numpy as np
import seaborn as sns; sns.set()
import pandas as pd
import pickle
import scipy
import pylab as plt
from scipy import sparse
import sklearn
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
import psutil
import time
import random



In [ ]:

rows_skip = np.array(random.sample(np.arange(1,45769,1).tolist(),k=(45768-3000)))

In [ ]:
#Note: want to modify this cell to take smaller sample of data


#https://github.com/berenslab/rna-seq-tsne/blob/master/demo.ipynb

#Loading the data
#Replaced .. in file paths with /content/drive/MyDrive/tSNE
%%time

# Load the Allen institute data. This takes a few minutes

# This function is needed because using Pandas to load these files in one go
# can eat up a lot of RAM. So we are doing it in chunks, and converting each
# chunk to the sparse matrix format on the fly.
def sparseload(filenames):
    genes = []
    sparseblocks = []
    areas = []
    cells = []
    #Want to figure out how to take smaller sample in next 2 lines. used nrows but not random sample - find way to improve
    for chunk1,chunk2 in zip(pd.read_csv(filenames[0], chunksize=1000, index_col=0, na_filter=False, skiprows = rows_skip),
                             pd.read_csv(filenames[1], chunksize=1000, index_col=0, na_filter=False, skiprows=rows_skip)):
        if len(cells)==0:
            cells = np.concatenate((chunk1.columns, chunk2.columns))
            areas = [0]*chunk1.columns.size + [1]*chunk2.columns.size

        genes.extend(list(chunk1.index))
        sparseblock1 = sparse.csr_matrix(chunk1.values.astype(float))
        sparseblock2 = sparse.csr_matrix(chunk2.values.astype(float))
        sparseblock = sparse.hstack((sparseblock1,sparseblock2), format='csr')
        sparseblocks.append([sparseblock])
        print('.', end='', flush=True)
    print(' done')
    counts = sparse.bmat(sparseblocks)
    return (counts.T, np.array(genes), cells, np.array(areas))

filenames = ['/content/drive/MyDrive/tSNE/data/tasic-nature/mouse_VISp_2018-06-14_exon-matrix.csv',
             '/content/drive/MyDrive/tSNE/data/tasic-nature/mouse_ALM_2018-06-14_exon-matrix.csv']
counts, genes, cells, areas = sparseload(filenames)

genesDF = pd.read_csv('/content/drive/MyDrive/tSNE/data/tasic-nature/mouse_VISp_2018-06-14_genes-rows.csv')
ids     = genesDF['gene_entrez_id'].tolist()
symbols = genesDF['gene_symbol'].tolist()
id2symbol = dict(zip(ids, symbols))
genes = np.array([id2symbol[g] for g in genes])

clusterInfo = pd.read_csv('/content/drive/MyDrive/tSNE/data/tasic-nature/sample_heatmap_plot_data.csv')
goodCells  = clusterInfo['sample_name'].values
ids        = clusterInfo['cluster_id'].values
labels     = clusterInfo['cluster_label'].values
colors     = clusterInfo['cluster_color'].values

clusterNames  = np.array([labels[ids==i+1][0] for i in range(np.max(ids))])
clusterColors = np.array([colors[ids==i+1][0] for i in range(np.max(ids))])
clusters   = np.copy(ids) - 1

ind = np.array([np.where(cells==c)[0][0] for c in goodCells])
counts = counts[ind, :]

tasic2018 = {'counts': counts, 'genes': genes, 'clusters': clusters, 'areas': areas,
             'clusterColors': clusterColors, 'clusterNames': clusterNames}
counts = []
memory_usage = psutil.virtual_memory()
print(f"Memory Usage: {memory_usage}")

FileNotFoundError: ignored

In [ ]:
#https://github.com/berenslab/rna-seq-tsne/blob/master/demo.ipynb
print('Number of cells:', tasic2018['counts'].shape[0])
print('Number of cells from ALM:', np.sum(tasic2018['areas']==0))
print('Number of cells from VISp:', np.sum(tasic2018['areas']==1))
print('Number of clusters:', np.unique(tasic2018['clusters']).size)
print('Number of genes:', tasic2018['counts'].shape[1])
print('Fraction of zeros in the data matrix: {:.2f}'.format(
    tasic2018['counts'].size/np.prod(tasic2018['counts'].shape)))

Number of cells: 23822
Number of cells from ALM: 15413
Number of cells from VISp: 10068
Number of clusters: 133
Number of genes: 3000
Fraction of zeros in the data matrix: 0.20


In [ ]:
#https://github.com/berenslab/rna-seq-tsne/blob/master/demo.ipynb
#Pre-processing:

%%time

# Feature selection

def nearZeroRate(data, threshold=0):
    zeroRate = 1 - np.squeeze(np.array((data>threshold).mean(axis=0)))
    return zeroRate

def meanLogExpression(data, threshold=0, atleast=10):
    nonZeros = np.squeeze(np.array((data>threshold).sum(axis=0)))
    N = data.shape[0]
    A = data.multiply(data>threshold)
    A.data = np.log2(A.data)
    meanExpr = np.zeros(data.shape[1]) * np.nan
    detected = nonZeros >= atleast
    meanExpr[detected] = np.squeeze(np.array(A[:,detected].mean(axis=0))) / (nonZeros[detected]/N)
    return meanExpr

def featureSelection(meanLogExpression, nearZeroRate, yoffset=.02, decay=1.5, n=3000):
    low = 0; up=10
    nonan = ~np.isnan(meanLogExpression)
    xoffset = 5
    for step in range(100):
        selected = np.zeros_like(nearZeroRate).astype(bool)
        selected[nonan] = nearZeroRate[nonan] > np.exp(-decay*meanLogExpression[nonan] + xoffset) + yoffset
        if np.sum(selected) == n:
            break
        elif np.sum(selected) < n:
            up = xoffset
            xoffset = (xoffset + low)/2
        else:
            low = xoffset
            xoffset = (xoffset + up)/2
    return selected

x = meanLogExpression(tasic2018['counts'], threshold=32)  # Get mean log non-zero expression of each gene
y = nearZeroRate(tasic2018['counts'], threshold=32)       # Get near-zero frequency of each gene
selectedGenes = featureSelection(x, y, n=3000)            # Adjust the threshold to select 3000 genes

memory_usage2 = psutil.virtual_memory()
print(f"Memory Usage: {memory_usage2}")

Memory Usage: svmem(total=13613305856, available=10334969856, percent=24.1, used=2942918656, free=1907228672, active=545542144, inactive=10904911872, buffers=96116736, cached=8667041792, shared=28979200, slab=158228480)
CPU times: user 1.71 s, sys: 239 ms, total: 1.95 s
Wall time: 3.53 s


In [ ]:
#https://github.com/berenslab/rna-seq-tsne/blob/master/demo.ipynb

#Not quite sure what this is supposed to do.. I think PCA initialization?

%%time

counts3k = tasic2018['counts'][:, selectedGenes]  # Feature selection

librarySizes = tasic2018['counts'].sum(axis=1)    # Compute library sizes
CPM = counts3k / librarySizes * 1e+6              # Library size normalisation


logCPM = np.log2(CPM + 1)                         # Log-transformation
logCPM = np.asarray(logCPM)

model = PCA(n_components=50, svd_solver='full')

pca = model.fit(logCPM)  # PCA (TypeError occurs here because logCPM is not an array. Tried to use np.asarray( ) to convert it and IT WORKED!!!!)

flipSigns = np.sum(pca.components_, axis=1) < 0             # fix PC signs
X = pca.transform(logCPM)
X[:, flipSigns] *= -1

print('Shape of the resulting matrix:', X.shape, '\n')


#Principal component analysis
memory_usage3 = psutil.virtual_memory()
print(f"Memory Usage: {memory_usage3}")

Shape of the resulting matrix: (23822, 50) 

Memory Usage: svmem(total=13613305856, available=10460676096, percent=23.2, used=2817224704, free=2031325184, active=546709504, inactive=10771038208, buffers=96411648, cached=8668344320, shared=28983296, slab=158146560)
CPU times: user 38.9 s, sys: 4.33 s, total: 43.2 s
Wall time: 30.6 s


In [ ]:
#Running hessian
z = sklearn.manifold.locally_linear_embedding(X,
                                          n_neighbors = 30, #Not sure what this is supposed to be ; set it to 30 because that's the t-SNE perplexity
                                          n_components = 6, #used equation n_neighbors > n_components * (n_components + 3) / 2, where n_neighbors = 30
                                          reg=0.001,
                                          eigen_solver='dense', #failed with it was 'auto' and error message recommended changing this to dense
                                          tol=1e-06, max_iter=100,
                                          method='hessian',
                                          hessian_tol=0.0001,
                                          modified_tol=1e-12,
                                          random_state=None,
                                          n_jobs=None)

#https://github.com/berenslab/rna-seq-tsne/blob/master/demo.ipynb
%matplotlib inline

plt.figure(figsize=(5,5))
plt.gca().set_aspect('equal', adjustable='datalim')
plt.scatter(z[:,0], z[:,1], s=1,
            color=tasic2018['clusterColors'][tasic2018['clusters']])
plt.title('t-SNE, early exaggeration = 4')
plt.tight_layout()
plt.show()

memory_usage4 = psutil.virtual_memory()

KeyboardInterrupt: ignored

In [ ]:
print(f"Memory Usage: {memory_usage4}")